### CUDA 버전 확인하기

- !nvidia-smi
- !nvcc --version

둘의 차이점은
nvidia-smi는 해당 장치에서 설치 가능한 가장 높은 버전을 보여주고,
nvcc --version은 현재 설치된 cuda 버전을 보여줌

출처 : https://stackoverflow.com/questions/9727688/how-to-get-the-cuda-version

In [ ]:
gpu_info = !nvcc --version

In [ ]:
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# PyTorch 2.x 버전 설치
try:
    # 기본적으로 https://pytorch.kr/get-started/locally/ 에서
    # cuda 버전과 패키지매니저에 맞는 설치 명령어를 확인 가능
    # %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    import torch
    print(torch.__version__)
except:
    pass

2.0.1+cu118


----------------------------------------------------------

# VGGNet 직접 구현하기

In [ ]:
# pytorch에서 사용할 함수들 호출하기
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [ ]:
# 데이터가 상당히 크기 때문에 시간이 좀 소요됨
transform = transforms.Compose([
    transforms.Resize(72),    # 크기를 줄여줌.
    transforms.RandomCrop(56),  # 랜덤으로 잘라줌.
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5 ), (0.5, 0.5, 0.5 )),
])

train_dataset = datasets.STL10(root='./data', split='train', download=True, transform=transform)
# train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)

test_dataset = datasets.STL10(root='./data', split='test', download=True, transform=transform)
# test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=256, shuffle=False)

In [ ]:
train_dataset[0][0]

tensor([[[ 0.1216,  0.1294,  0.1294,  ...,  0.2392,  0.2627,  0.2784],
         [ 0.1137,  0.1294,  0.1294,  ...,  0.2471,  0.2784,  0.2941],
         [ 0.1137,  0.1294,  0.1294,  ...,  0.2627,  0.2784,  0.2941],
         ...,
         [ 0.0745,  0.1059,  0.1216,  ...,  0.1373,  0.1451,  0.1529],
         [ 0.0824,  0.1216,  0.1451,  ...,  0.2078,  0.1765,  0.1137],
         [ 0.1059,  0.1294,  0.1529,  ...,  0.2235,  0.2314,  0.1843]],

        [[ 0.0980,  0.1059,  0.1059,  ...,  0.1373,  0.1608,  0.1765],
         [ 0.0902,  0.1059,  0.1059,  ...,  0.1294,  0.1529,  0.1686],
         [ 0.0902,  0.1059,  0.1059,  ...,  0.1373,  0.1529,  0.1686],
         ...,
         [-0.0353, -0.0196, -0.0039,  ...,  0.0980,  0.0980,  0.1059],
         [-0.0118,  0.0039,  0.0196,  ...,  0.1765,  0.1373,  0.0588],
         [-0.0039,  0.0118,  0.0275,  ...,  0.1922,  0.1922,  0.1529]],

        [[-0.1608, -0.1529, -0.1529,  ..., -0.1294, -0.1059, -0.0745],
         [-0.1686, -0.1529, -0.1529,  ..., -0

In [ ]:
class VGGNet11(nn.Module):

    def __init__(self, num_classes=10):
        super(VGGNet11, self).__init__()
        self.convnet = nn.Sequential(
            # 여기에 CNN 모델을 구현해주세요.
            # 파라미터가 많아지면 헷갈리기 때문에, 파라미터 이름을 주는것도 좋음
            # input image size = 3, 56, 56  위에서 랜덤크롭해서 56으로 잘라줘서  channel 수 , image size * image size
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            # 64, 56, 56
            nn.ReLU(),          # ReLU(True) 넣어줘도 상관없음. 넣으면 inplace=True 여서 더 좋긴함.
            nn.MaxPool2d(2),    # 2 는 MaxPool 커널사이즈 2*2
            # 64, 28, 28

            nn.Conv2d(64, 128, 3, 1,1),
            # 128, 28, 28
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 128, 14, 14

            nn.Conv2d(128, 256, 3, 1,1),
            # 256, 14, 14
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            # 256, 14, 14
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 256, 7, 7

            nn.Conv2d(256, 512, 3, 1,1),
            # 512, 7, 7
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            # 512, 7, 7
            nn.ReLU(),
            nn.MaxPool2d(2),
            # 512, 3, 3

            nn.Conv2d(512, 512, 3, 1, 1),
            # 512, 3, 3
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            # 512, 3, 3
            nn.ReLU(),
            nn.MaxPool2d(2)
            # 512, 1, 1
        )

        self.fclayer = nn.Sequential(
            # 여기에 FC 모델을 구현해주세요.
            nn.Flatten(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.convnet(x)
        x = torch.flatten(x, 1)
        output = self.fclayer(x)
        return output

In [ ]:
class VGGNet13(nn.Module):
    def __init__(self,) :
        super(VGGNet13, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(),

        )

    def forward(self, x):

        return x

In [ ]:
model = VGGNet11(num_classes=10)

In [ ]:
model

VGGNet11(
  (convnet): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU()
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (16): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding

In [ ]:
print('{:.6f}'.format(5))

5.000000


In [ ]:
# 학습을 위한 설정
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(model.parameters(),lr=1e-5)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

In [ ]:
device

device(type='cuda')

In [ ]:
# 모델 학습
epochs = 2
dry_run = False # 1 배치만 훈련

for epoch in range(1, epochs+1):
    # 학습
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.detach().cpu().item()))
            if dry_run:
                break

    # 테스트
    model.eval()
    test_loss = 0
    correct = 0

    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        with torch.no_grad():
            output = model(data)
        test_loss += criterion(output, target).detach().cpu().item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    scheduler.step()


Train Epoch: 1 [0/5000 (0%)]	Loss: 2.302947
Train Epoch: 1 [2560/5000 (50%)]	Loss: 2.302079

Test set: Average loss: 0.0092, Accuracy: 800/8000 (10%)

Train Epoch: 2 [0/5000 (0%)]	Loss: 2.302706
Train Epoch: 2 [2560/5000 (50%)]	Loss: 2.302049

Test set: Average loss: 0.0092, Accuracy: 897/8000 (11%)



In [ ]:
for i, (name, param) in enumerate(model.named_parameters()):
    print(name)
    print(param.shape)
    if i == 3:
      break

convnet.0.weight
torch.Size([64, 3, 3, 3])
convnet.0.bias
torch.Size([64])
convnet.3.weight
torch.Size([128, 64, 3, 3])
convnet.3.bias
torch.Size([128])
